In [1]:
import gzip
import json
import random
import sklearn
import string
from collections import defaultdict
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.manifold import TSNE

In [2]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    for l in f:
        d = json.loads(l)
        if d['rating'] != None:
            yield int(d['rating']), d['review_text']

data = []
for rating, review in readCSV('renttherunway_final_data.json.gz'):
    data.append((rating,review))
    
train = data[:int(len(data) * 0.8)]
test = data[int(len(data) * 0.8):]

In [4]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for _,review in train:
    r = ''.join([c for c in review.lower() if not c in punctuation])
    ws = r.split()
    ws2 = [''.join(x) for x in list(zip(ws[:-1],ws[1:]))]
    #ws3 = [' '.join(x) for x in list(zip(ws[:-2],ws[1:-1],ws[2:]))]
    #ws4 = [' '.join(x) for x in list(zip(ws[:-3],ws[1:-2],ws[2:-1],ws[3:]))]
    #ws5 = [' '.join(x) for x in list(zip(ws[:-4],ws[1:-3],ws[2:-2],ws[3:-1],ws[4:]))]
    for w in ws+ws2:
        wordCount[w] += 1

In [5]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
words = [x[1] for x in counts[:1000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [6]:
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum.lower() if not c in punctuation])
    ws = r.split()
    ws2 = [''.join(x) for x in list(zip(ws[:-1],ws[1:]))]
    #ws3 = [' '.join(x) for x in list(zip(ws[:-2],ws[1:-1],ws[2:]))]
    #ws4 = [' '.join(x) for x in list(zip(ws[:-3],ws[1:-2],ws[2:-1],ws[3:]))]
    #ws5 = [' '.join(x) for x in list(zip(ws[:-4],ws[1:-3],ws[2:-2],ws[3:-1],ws[4:]))]
    for w in ws+ws2:
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) # offset
    return feat

In [7]:
X = [feature(review) for _,review in train]
y = [r for r,_ in train]

In [8]:
# Regularized regression
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
theta = clf.coef_
X_test = [feature(review) for _,review in test]
y_test = [r for r,_ in test]
predictions = clf.predict(X_test)

In [9]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [10]:
MSE(predictions, y_test)

1.5129230071862318